In [9]:
# examples with classes

class Calculator:
    def __init__(self, name):
        self.name = name  # Store data in this instance
    
    def add(self, a, b):
        print(f"{self.name} is adding {a} + {b}")  # Access self.name
        return a + b

# Usage:
calc = Calculator("MyCalc")
summy = calc.add(2, 3)  # Python automatically passes calc as self
print(f"The computed sum is {summy}\n")

class Duck:
    def __init__(self, name):
        self.name = name  # Store the duck's name
    
    def introduce(self):
        print(f"Quack! My name is {self.name}")
    
    def quack(self):
        print("Quack!")

# Usage examples:
donald = Duck("Donald")
donald.introduce()
donald.quack()

MyCalc is adding 2 + 3
The computed sum is 5

Quack! My name is Donald
Quack!


In [ ]:
# gradient descent basic example.
# Not clear how "non-analytical" functions can get until torch stops working...
# copilot claims torch ONLY uses analytical stuff, even for eigenvalues...??
# general goot practice: prefer torch functions

import numpy as np
import torch

#possible cost functions
def cost_function_1(x):
    return x[0]**2 + x[1]**4 + (x[0] + x[1])**6 + x[2]**2 + x[3]**2

def cost_function_2(x): # with eigenvalues involved
    # Reshape x into a matrix
    A = x.view(2, 2)  # If x has 4 elements
    eigenvals = torch.linalg.eigvals(A)
    eigenvals_real = torch.real(eigenvals)
    return torch.sum(eigenvals_real**2)

def cost_function_3(x):
    return max(x)

d = 2
cost_function = cost_function_1
# define variable to be optimized
x = torch.tensor([1.0, -2.0, 5., 3.14], dtype=torch.double, requires_grad=True)
# define optimizer
learning_rate = 0.01
n_steps = 10000
optimizer = torch.optim.Adam([x], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(x)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 1000 == 0:
        print(f"x = {x.tolist()}, cost function = {loss}")


x = [1.009999999975, -1.9900000000026317, 4.99000000001, 3.1300000000159236], cost function = 52.8596
x = [0.005897630008671893, -0.2904937204018134, 0.13457109604084116, 0.001668087453219903], cost function = 0.026028579594744642
x = [0.005897630008671893, -0.2904937204018134, 0.13457109604084116, 0.001668087453219903], cost function = 0.026028579594744642
x = [0.00037449115354536817, -0.16550934894657934, 3.226168500859898e-05, 3.5483309608153823e-12], cost function = 0.0007721943108706685
x = [0.00037449115354536817, -0.16550934894657934, 3.226168500859898e-05, 3.5483309608153823e-12], cost function = 0.0007721943108706685
x = [5.4498576897916806e-05, -0.11256823123990456, 2.5927969375248464e-12, 7.114101782273717e-33], cost function = 0.00016282688486587434
x = [5.4498576897916806e-05, -0.11256823123990456, 2.5927969375248464e-12, 7.114101782273717e-33], cost function = 0.00016282688486587434
x = [1.097225479559197e-05, -0.08172503814583272, -3.4529096537761817e-28, 1.1222778471653

In [1]:
import sys
import os
import numpy as np
import torch

# For Jupyter notebooks, use "relative path"
sys.path.insert(0, os.path.join('..', 'src'))

from kaustav_conj.utils import h, H, nK, block_spec, M_to_A
from kaustav_conj.core import build_cost_function

n = [0.2, 0.4]
lamb = 1
# optimal rotation is e.g. a hadamard, yielding maximally mixed block spectrum [0.3, 0.3]
# so optimal M should be s.t. M_to_A(M) = log(hadamard). here is one possibility:
# M_optimal = torch.tensor([[ 0.4601,  0.0000], [-1.1107,  2.6815]], dtype=torch.double, requires_grad=True)

cost_function = build_cost_function(n, lamb)

# initialize variable to be optimized: here are some choices
# M = torch.tensor([[0., 0.1], [0., 0.]], dtype=torch.double, requires_grad=True) # start with U0 close to identity
# M = M_optimal
M = torch.rand(2, 2) - 0.5 * torch.ones(2,2)
M.requires_grad_(True)

# define optimizer
learning_rate = 0.01
n_steps = 3001
optimizer = torch.optim.Adam([M], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(M)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 1000 == 0:
        print(f"Step # {step}")
#        print(f"M = {M.tolist()}, cost function = {loss}")

# print optimized unitary and block spec
U_best = torch.matrix_exp(M_to_A(M))
D = torch.diag(torch.tensor(n, dtype=torch.cdouble))
b_best = torch.real(block_spec(U_best @ D @ U_best.adjoint(), lamb))
H = torch.tensor([[1., 1.], [1., -1.]]/np.sqrt(2), dtype=torch.double)
print(f"\n The optimized unitary is \n {U_best.data} \n ")
print(f"Absolute values of its entries: \n {torch.abs(U_best).data} \n ")
print(f"Compare to Hadamard: \n {H.data} \n ")
print(f"The optimized block spectrum is \n {b_best.data}")



Step # 0
Step # 1000
Step # 2000
Step # 3000

 The optimized unitary is 
 tensor([[ 0.7049-0.0557j, -0.6732-0.2165j],
        [ 0.6198-0.3403j,  0.7027-0.0791j]], dtype=torch.complex128) 
 
Absolute values of its entries: 
 tensor([[0.7071, 0.7071],
        [0.7071, 0.7071]], dtype=torch.float64) 
 
Compare to Hadamard: 
 tensor([[ 0.7071,  0.7071],
        [ 0.7071, -0.7071]], dtype=torch.float64) 
 
The optimized block spectrum is 
 tensor([0.3000, 0.3000], dtype=torch.float64)


In [ ]:
d = 6
n = [0.1, 0.2, 0.3, 0.32, 0.6, 0.9]
lamb = 4
n_conj = nK(n, lamb)

cost_function = build_cost_function(n, lamb)

# initialize variable to be optimized
M = torch.rand(d, d) - 0.5 * torch.ones(d,d)
M.requires_grad_(True)

# define optimizer
learning_rate = 0.01
n_steps = 3001
optimizer = torch.optim.Adam([M], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(M)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 1000 == 0:
        print(f"Step # {step}")
#        print(f"M = {M.tolist()}, cost function = {loss}")

# print optimized (unitary and) block spec
U_best = torch.matrix_exp(M_to_A(M))
D = torch.diag(torch.tensor(n, dtype=torch.cdouble))
b_best = torch.real(block_spec(U_best @ D @ U_best.adjoint(), lamb))
b_best_sorted = torch.round(torch.cat([
    torch.sort(b_best[:lamb], descending=True).values,
    torch.sort(b_best[lamb:], descending=True).values
]), decimals=5).tolist()
# print(f"\n The optimized unitary is \n {U_best.data} \n ")
print(f"The optimized block spectrum is \n {b_best_sorted}")
print(f"Compare with the conjectured value: \n {n_conj}")

Step # 0
Step # 1000
Step # 2000
Step # 3000


TypeError: round() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (Tensor input, *, Tensor out)
 * (Tensor input, *, int decimals, Tensor out)


In [16]:
n_conj = nK(torch.tensor(n, dtype=torch.double), lamb)
n_conj

[tensor(0.5000, dtype=torch.float64),
 tensor(0.4000, dtype=torch.float64),
 tensor(0.3200, dtype=torch.float64),
 tensor(0.3000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.4000, dtype=torch.float64)]